In [1]:
%matplotlib inline
#imports
import os
import random
import pandas as pd
import numpy as np
import cv2
import shutil
import collections
from matplotlib import pyplot as plt
from sentence_transformers import SentenceTransformer
#model = SentenceTransformer('bert-base-nli-mean-tokens')

In [2]:
dataPath = "../Data/MET/"
GANDataPath = "../Data/CelebA"
data = pd.read_csv("../Data/MET/works_downloaded.csv", sep="\t")

In [3]:
data["Tags"].fillna("", inplace=True)

In [4]:
data["TagList"] = data["Tags"].apply(lambda x : x.lower().split("|"))

In [5]:
data.drop(data[data["Filename"].duplicated()].index.tolist(), inplace=True)

In [26]:
def get_top_tags(data, thresh=20):
    count = collections.Counter()
    for taglist in data["TagList"].tolist():
        for tag in taglist:
            count[tag] += 1
    top_tags = []
    for tag in count.most_common():
        if tag[1] >= thresh:
            top_tags.append(tag[0])
    return top_tags, count

In [30]:
#top_tags, count = get_top_tags(data, thresh=20)

In [31]:
# for tag, val in count.most_common():
#     print("|" + tag + "|" + str(val) + "|")

In [ ]:
#top_tags[12]

In [8]:
def tag_vector(top_tags, tags):
    tagvect = np.zeros(len(top_tags))
    for tag in tags:
        if tag in top_tags:
            i = top_tags.index(tag)
            tagvect[i] = 1
    return tagvect

In [9]:
data["TagVect"] = data["TagList"].apply(lambda x: tag_vector(top_tags, x))

In [ ]:
# domCol = get_dom_col(dataPath, 'ap40.144.jpg', quant=20)
# colImg = (np.ones((500, 500, 3)) * domCol * 255).astype('uint8')
# plt.axis("off")
# colImg = cv2.cvtColor(colImg, cv2.COLOR_RGB2BGR)
# cv2.imwrite("col.jpg", colImg)
# plt.imshow(colImg)

In [33]:
len(data)

1781

In [32]:
data[data["Filename"] == 'ap40.144.jpg']

,Filename,Title,Artist,Medium,Tags,MET_Object_ID,Description,TagList,TagVect
19,ap40.144.jpg,Mrs. William Loring Andrews,George Augustus Baker Jr.,Oil on canvas,Portraits|Women,10093,This portrait has an unusual oak frame that ap...,"[portraits, women]","[0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [16]:
'''
Filter data for those matching some set of tags
'''
def get_file_list_for_tag(tags):
    filt = data["TagList"].apply(lambda x: np.any([tag in x for tag in tags]))
    return data[filt]

'''
Copy filtered files
'''
def copy_tagged(tagged, num):
    i = 0
    domCols = np.zeros((num, 3))
    for filename in tagged["Filename"].tolist()[:num]:
        if len(os.listdir(GANDataPath)) < num:
            shutil.copy(os.path.join(dataPath, filename), 
                        os.path.join(GANDataPath, "img_align_celeba", str(i).zfill(4) + ".jpg"))
            domCols[i] = get_dom_col(dataPath, filename)
            i += 1
    np.savetxt(os.path.join(GANDataPath, "DomCols.txt"), domCols, fmt='%f')
    np.savetxt(os.path.join(GANDataPath, "TagVects.txt"), tagged["TagVect"].tolist()[:num], fmt='%u') 
    
            
        
'''
randomly pick an image
'''
def pick_img(dataPath, tagged):
    randFile = random.choice(tagged['Filename'].tolist())
    img = cv2.imread(os.path.join(dataPath, randFile))
    return img

'''
get image at an index
'''
def get_img(dataPath, tagged, i):
    file = tagged['Filename'].tolist()[i]
    img = cv2.imread(os.path.join(dataPath, file))
    return img

'''
Use numpy bincount to get the most common bgr color in the image
'''
def bincount(img):
    img2D = img.reshape(-1, img.shape[-1])
    col_range = (256, 256, 256) # generically : a2D.max(0)+1
    img1D = np.ravel_multi_index(img2D.T, col_range)
    return np.unravel_index(np.bincount(img1D).argmax(), col_range)

'''
Given a filename and a quantization factor, read in the corresponding image,
quantize it, fidn the dominant color of the 
'''
def get_dom_col(dataPath, imgname, quant=20):
    img = cv2.imread(os.path.join(dataPath, imgname))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_quantized = img // quant
    img_quantized *= quant
    domCol = bincount(img_quantized)
    return np.array(domCol, "uint8") / ((255//quant) * quant)

In [ ]:
p = get_file_list_for_tag(['portraits', 'men', 'women', 'boys', 'girls', 'angels', 'children', 'nudes'])

In [19]:
#len(p)
copy_tagged(data, 1776)

In [13]:
len(data)

1781

In [ ]:
len(p["Filename"].tolist()[:1184])

In [ ]:
p["Title"]

In [ ]:
img = get_img(dataPath, p, 0)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)

In [ ]:
img_quantized = img // 20
img_quantized *= 20
plt.imshow(img_quantized)

In [ ]:
domCol = bincount_app(img_quantized)

In [ ]:
np.array(domCol, "uint8")

In [ ]:
colImg = np.ones_like(img) * domCol
plt.imshow(colImg)

Sent embedding stuff below:

In [35]:
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [37]:
data["Title Embedding"] = data["Title"].apply(model.encode)

In [45]:
np.savetxt(os.path.join(GANDataPath, "TitleVects.txt"), data["Title Embedding"].values.tolist()[:1776], fmt='%f')

In [46]:
data["Title Embedding"].iloc[0].shape

(768,)